[View in Colaboratory](https://colab.research.google.com/github/0xc0d3r/startx/blob/master/Naval_mine_identifier.ipynb)

**Source:**

The data set was contributed to the benchmark collection by Terry Sejnowski, now at the Salk Institute and the University of California at San Deigo. The data set was developed in collaboration with R. Paul Gorman of Allied-Signal Aerospace Technology Center.

**Data Set Information:**

The file "sonar.mines" contains 111 patterns obtained by bouncing sonar signals off a metal cylinder at various angles and under various conditions. The file "sonar.rocks" contains 97 patterns obtained from rocks under similar conditions. The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock.

Each pattern is a set of 60 numbers in the range 0.0 to 1.0. Each number represents the energy within a particular frequency band, integrated over a certain period of time. The integration aperture for higher frequencies occur later in time, since these frequencies are transmitted later during the chirp.

The label associated with each record contains the letter "R" if the object is a rock and "M" if it is a mine (metal cylinder). The numbers in the labels are in increasing order of aspect angle, but they do not encode the angle directly.

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

In [0]:
#Loading the data
data = pd.read_csv('https://raw.githubusercontent.com/malinenimaurya/datasets/master/Sonar.csv')

In [126]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1


In [127]:
data[data.columns[60]].value_counts()

0    111
1     97
Name: Class, dtype: int64

 **Input and Output**

In [0]:
input_data = data[data.columns[0:60]].values
output = data[data.columns[60]].values

In [129]:
print(input_data.shape)

(208, 60)


In [0]:
#One hot encoding the target variable
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [0]:
output_data = one_hot_encode(output)

In [132]:
#Need to shuffle as the all Rocks and Mines are together
input_data,output_data = shuffle (input_data, output_data, random_state = 0)
print(input_data)

[[0.0079 0.0086 0.0055 ... 0.0058 0.0059 0.0032]
 [0.0229 0.0369 0.004  ... 0.0029 0.0104 0.0163]
 [0.0442 0.0477 0.0049 ... 0.0105 0.0059 0.0105]
 ...
 [0.0228 0.0106 0.013  ... 0.0045 0.0063 0.0039]
 [0.0373 0.0281 0.0232 ... 0.0054 0.0085 0.006 ]
 [0.018  0.0444 0.0476 ... 0.005  0.0073 0.0022]]


In [0]:
# Splittting the data into train and test
train_x, test_x, train_y, test_y = train_test_split(input_data, output_data, test_size = 0.20, random_state = 0)

In [0]:
#Code for the Artificial neural net

keep_prob = tf.placeholder(tf.float32)



In [0]:
X = tf.placeholder(tf.float32, [None, 60])
W1 = tf.Variable(tf.truncated_normal([60, 256], stddev=0.1))
b1 = tf.Variable(tf.ones([256])/10)

W2 = tf.Variable(tf.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.ones([128])/10)

W3 = tf.Variable(tf.truncated_normal([128, 64], stddev=0.1))
b3 = tf.Variable(tf.ones([64])/10)

W4 = tf.Variable(tf.truncated_normal([64, 2], stddev=0.1))
b4 = tf.Variable(tf.ones([2])/10)

Y1 = tf.nn.relu(tf.matmul(X,W1)+b1)
Y1d = tf.nn.dropout(Y1, keep_prob)

Y2 = tf.nn.relu(tf.matmul(Y1d,W2)+b2)
Y2d = tf.nn.dropout(Y2, keep_prob)

Y3 = tf.nn.relu(tf.matmul(Y2d,W3)+b3)
Y3d = tf.nn.dropout(Y3, keep_prob)

YLogits = tf.matmul(Y3d,W4)+b4
Y = tf.nn.softmax(YLogits)


In [0]:
Y_ = tf.placeholder(tf.float32, [None, 2])
# loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=YLogits, labels=Y_) # handled nan(log0) for cross entropy, different optimisers
cross_entropy = tf.reduce_mean(cross_entropy)*100


# % of correct answers found in batch
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [0]:
optimizer = tf.train.AdamOptimizer(0.003)         #can use RMSprop, Adadelta, gradientdescent etc
train_step = optimizer.minimize(cross_entropy)

In [142]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
accuracy_history = []
for i in range(1000):
  train_data={X: train_x, Y_: train_y, keep_prob : 0.7}
  # train
  sess.run(train_step, feed_dict=train_data)
  if i%50 == 0:
    a,c = sess.run([accuracy, cross_entropy], feed_dict=train_data)
    accuracy_history.append(a)
    print("iteration:"+ str(i) + ", accuracy:" + str(a) +", cross_entropy:"+ str(c))
  

iteration:0, accuracy:0.55421686, cross_entropy:68.25511
iteration:50, accuracy:0.8072289, cross_entropy:37.82855
iteration:100, accuracy:0.96385545, cross_entropy:16.001583
iteration:150, accuracy:0.9819277, cross_entropy:5.452175
iteration:200, accuracy:0.9879518, cross_entropy:3.0582836
iteration:250, accuracy:0.9879518, cross_entropy:2.373193
iteration:300, accuracy:1.0, cross_entropy:0.29304332
iteration:350, accuracy:1.0, cross_entropy:0.52140516
iteration:400, accuracy:1.0, cross_entropy:0.46745342
iteration:450, accuracy:1.0, cross_entropy:0.06272124
iteration:500, accuracy:1.0, cross_entropy:0.19166951
iteration:550, accuracy:1.0, cross_entropy:0.03059021
iteration:600, accuracy:1.0, cross_entropy:0.32274273
iteration:650, accuracy:1.0, cross_entropy:0.022781126
iteration:700, accuracy:1.0, cross_entropy:0.3668615
iteration:750, accuracy:1.0, cross_entropy:0.1228814
iteration:800, accuracy:1.0, cross_entropy:0.33106214
iteration:850, accuracy:1.0, cross_entropy:0.33446702
iter

In [143]:
test_data={X: test_x, Y_: test_y, keep_prob : 1}
a,c = sess.run([accuracy, cross_entropy], feed_dict=test_data)
print(a,c)

0.88095236 79.26053
